In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [2]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

# Paths
image_dir = '/kaggle/input/dog-and-cat-detection/images'
annotation_dir = '/kaggle/input/dog-and-cat-detection/annotations'
output_label_dir = '/kaggle/working/labels'

os.makedirs(output_label_dir, exist_ok=True)

# Class name to ID mapping
class_map = {'cat': 0, 'dog': 1}  # Modify as needed

# Loop over annotation files
for xml_file in os.listdir(annotation_dir):
    if not xml_file.endswith('.xml'):
        continue

    tree = ET.parse(os.path.join(annotation_dir, xml_file))
    root = tree.getroot()

    image_filename = root.find('filename').text
    image_path = os.path.join(image_dir, image_filename)

    img = Image.open(image_path)
    img_w, img_h = img.size

    label_filename = os.path.splitext(image_filename)[0] + '.txt'
    label_path = os.path.join(output_label_dir, label_filename)

    with open(label_path, 'w') as f:
        for obj in root.findall('object'):
            class_name = obj.find('name').text.lower()
            class_id = class_map.get(class_name)
            if class_id is None:
                continue

            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / img_w
            y_center = ((ymin + ymax) / 2) / img_h
            width = (xmax - xmin) / img_w
            height = (ymax - ymin) / img_h

            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


In [4]:
import shutil
from sklearn.model_selection import train_test_split

# Get all images
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# Train/val split
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Create folders
for split in ['train', 'val']:
    os.makedirs(f'/kaggle/working/images/{split}', exist_ok=True)
    os.makedirs(f'/kaggle/working/labels/{split}', exist_ok=True)

# Copy files
for split, files in [('train', train_files), ('val', val_files)]:
    for img_file in files:
        base = os.path.splitext(img_file)[0]
        shutil.copy(os.path.join(image_dir, img_file), f'/kaggle/working/images/{split}/{img_file}')
        shutil.copy(os.path.join(output_label_dir, base + '.txt'), f'/kaggle/working/labels/{split}/{base}.txt')


In [5]:
import yaml

data = {
    'train': '/kaggle/working/images/train',
    'val': '/kaggle/working/images/val',
    'nc': 2,
    'names': ['cat', 'dog']
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)


In [6]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs=20, imgsz=640)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 79.7MB/s]


Ultralytics 8.3.157 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=Fals

100%|██████████| 755k/755k [00:00<00:00, 16.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        738        738      0.973      0.988      0.994        0.9



20 epochs completed in 0.210 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.157 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:04<00:00,  4.92it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        738        738      0.974      0.989      0.994        0.9
                   cat        231        231       0.98      0.987      0.995      0.937
                   dog        507        507      0.967       0.99      0.993      0.864
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f956ebc3950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [7]:
from ultralytics import YOLO

model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')  # adjust path if needed
results = model.predict(source='/kaggle/working/images/val', save=True, imgsz=640)



image 1/738 /kaggle/working/images/val/Cats_Test1009.png: 448x640 1 dog, 36.3ms
image 2/738 /kaggle/working/images/val/Cats_Test1022.png: 480x640 1 dog, 36.3ms
image 3/738 /kaggle/working/images/val/Cats_Test1023.png: 480x640 1 cat, 6.1ms
image 4/738 /kaggle/working/images/val/Cats_Test1025.png: 448x640 1 dog, 6.7ms
image 5/738 /kaggle/working/images/val/Cats_Test1038.png: 640x480 1 cat, 34.6ms
image 6/738 /kaggle/working/images/val/Cats_Test1040.png: 416x640 1 cat, 35.4ms
image 7/738 /kaggle/working/images/val/Cats_Test1046.png: 640x448 1 dog, 34.6ms
image 8/738 /kaggle/working/images/val/Cats_Test1070.png: 448x640 1 cat, 6.9ms
image 9/738 /kaggle/working/images/val/Cats_Test1087.png: 448x640 1 dog, 6.1ms
image 10/738 /kaggle/working/images/val/Cats_Test1089.png: 480x640 1 dog, 6.9ms
image 11/738 /kaggle/working/images/val/Cats_Test109.png: 640x480 1 cat, 1 dog, 6.7ms
image 12/738 /kaggle/working/images/val/Cats_Test1098.png: 640x448 1 cat, 6.6ms
image 13/738 /kaggle/working/images/v

In [8]:
model.export(format='onnx')  # or 'torchscript', 'coreml', etc.


Ultralytics 8.3.157 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/

PyTorch: starting from '/kaggle/working/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.9 MB)
requirements: Ultralytics requirements ['onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 4.2s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success ✅ 6.2s, saved as '/kaggle/working/runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (6.5s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train/weights/best.onnx imgsz=6

'/kaggle/working/runs/detect/train/weights/best.onnx'